In [1]:
import sys

import numpy as np
import pandas as pd

src_dir = '../src/'
if src_dir not in sys.path:
    sys.path.append(src_dir)
    
from data_processing import extract_annotations
from metrics import calculate_conf, IoU

INFO     - The mmdet config folder already exists. No need to downloaded it. Path : /home/chris/.icevision/mmdetection_configs/mmdetection_configs-2.16.0/configs | icevision.models.mmdet.download_configs:download_mmdet_configs:17


In [2]:
DATA_DIR = '../../data'

# Load in dataframe and do the standard processing
df = pd.read_csv(f'{DATA_DIR}/train.csv')
df['annotations'] = df['annotations'].map(eval)
df['num_annotations'] = df['annotations'].map(len)

# Extract the invidual bounding boxes and their annotation columns
dfexp = df.explode('annotations')
dfexp = extract_annotations(dfexp)

In [3]:
dfexp.head()

,video_id,sequence,video_frame,sequence_frame,image_id,...,annotations_processed,xmin,ymin,bb_width,bb_height
0,0,40258,0,0,0-0,...,"{'xmin': -1, 'ymin': -1, 'bb_width': -1, 'bb_height': -1}",-1,-1,-1,-1
1,0,40258,1,1,0-1,...,"{'xmin': -1, 'ymin': -1, 'bb_width': -1, 'bb_height': -1}",-1,-1,-1,-1
2,0,40258,2,2,0-2,...,"{'xmin': -1, 'ymin': -1, 'bb_width': -1, 'bb_height': -1}",-1,-1,-1,-1
3,0,40258,3,3,0-3,...,"{'xmin': -1, 'ymin': -1, 'bb_width': -1, 'bb_height': -1}",-1,-1,-1,-1
4,0,40258,4,4,0-4,...,"{'xmin': -1, 'ymin': -1, 'bb_width': -1, 'bb_height': -1}",-1,-1,-1,-1


In [4]:
# def IoU(box1, box2):
#     """ Calculate the IoU of a box pair
#     box1: (xmin1, ymin1, width1, height1)
#     box2: (xmin2, ymin2, width2, height2)
#     """
#     # Extract params
#     xmin1, ymin1, width1, height1 = box1
#     xmin2, ymin2, width2, height2 = box2
    
#     # Compute the x, y maxes
#     xmax1 = xmin1 + width1 - 1
#     xmax2 = xmin2 + width2 - 1
    
#     ymax1 = ymin1 + height1 - 1
#     ymax2 = ymin2 + height2 - 1
    
#     xA = min(xmax1, xmax2)
#     xB = max(xmin1, xmin2)
#     x_overlap = max(xA - xB + 1, 0)
    
#     yA = min(ymax1, ymax2)
#     yB = max(ymin1, ymin2)
#     y_overlap = max(yA - yB + 1, 0)
    
#     intersection = x_overlap * y_overlap
    
# #     area1 = (xmax1 - xmin1 + 1) * (ymax1 - ymin1 + 1)
# #     area2 = (xmax2 - xmin2 + 1) * (ymax2 - ymin2 + 1)

#     area1 = width1 * height1
#     area2 = width2 * height2
    
#     union = area1 + area2 - intersection
    
#     iou = intersection / union
    
#     return iou

In [6]:
%time IoU((1, 1, 5, 5), (2, 1, 5, 6), mode='xywh')

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 10.7 µs


0.5714285714285714

In [7]:
%time IoU((1, 1, 5, 5), (0, 3, 5, 4), mode='xywh')

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 11.4 µs


0.36363636363636365

In [8]:
%time IoU((1, 1, 5, 5), (1, 8, 5, 4), mode='xywh')

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 11.2 µs


0.0

In [9]:
%time IoU((8, 1, 5, 5), (1, 8, 5, 4), mode='xywh')

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 11 µs


0.0

Try the same thing with xmin, ymin, xmax, ymax

In [10]:
%time IoU((1, 1, 5, 5), (2, 1, 6, 6), mode='xyxy')

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 6.68 µs


0.5714285714285714

In [11]:
%time IoU((1, 1, 5, 5), (0, 3, 4, 6), mode='xyxy')

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 10.7 µs


0.36363636363636365

In [12]:
gts = np.array([(1, 1, 5, 5), (8, 1, 5, 5)])
preds = np.array([(2, 1, 5, 6), (0, 3, 5, 4), (1, 8, 5, 4), (1, 8, 5, 4)])
confs = np.array([0.7, 0.8, 0.5, 0.5])

confs_order = np.argsort(confs)[::-1]
preds = preds[confs_order]

In [13]:
preds

array([[0, 3, 5, 4],
       [2, 1, 5, 6],
       [1, 8, 5, 4],
       [1, 8, 5, 4]])

#### Create IoU matrix

In [14]:
%%time

main_ioumat = np.zeros((len(gts), len(preds)))

for i, gt in enumerate(gts):
    for j, pred in enumerate(preds):
        main_ioumat[i, j] = IoU(gt, pred, mode='xywh')

CPU times: user 116 µs, sys: 2 µs, total: 118 µs
Wall time: 121 µs


In [15]:
main_ioumat

array([[    0.36364,     0.57143,           0,           0],
       [          0,           0,           0,           0]])

In [12]:
%%time

# We'll need this at each threshold
preds_set = set([x for x in range(0, len(preds))])

tps = 0
fps = 0
fns = 0

for thr in np.arange(0.3, 0.85, 0.05):
#     print(thr)

    ioumat = main_ioumat.copy()
    ioumat[ioumat < thr] = 0
    # ioumat

    mask = (ioumat != 0)
    res = np.where(mask.any(axis=1), mask.argmax(axis=1), -1)

    tp_set = set(res[res != -1])

    tps += len(tp_set)
    fps += len(preds_set - tp_set)
    fns += len(res) - len(tp_set)

CPU times: user 330 µs, sys: 0 ns, total: 330 µs
Wall time: 265 µs


In [13]:
tps, fps, fns

(6, 38, 16)

In [14]:
calculate_conf(gts, preds, confs)

(6, 38, 16)

In [15]:
%%time

precision = tps / (tps + fps)
recall = tps / (tps + fns)

beta = 2
# f2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall)
f2 = 5 * precision * recall / (4 * precision + recall)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [16]:
f2

0.22727272727272724

### Load in model and try on test set

In [17]:
# from icevision.all import *

# MODEL_FILE = f'../../models/yolov5_large_p6_lr_1.0e-3_80pct_0.3_samplefrac_1_10_epochs.pth'

# model_type = models.ultralytics.yolov5
# backbone = model_type.backbones.large_p6

# model = model_type.model(backbone=backbone(pretrained=True), 
#                          num_classes=2,
#                          img_size = 832)

# state_dict = torch.load(MODEL_FILE, map_location=torch.device('cuda')) 

# model.load_state_dict(state_dict)

# model = model.cuda(0)

# test = pd.read_csv('../../data/test.csv')

# parser = ReefParser(template_record, f'{DATA_DIR}/train_images', data)
# train_records, valid_records = parser.parse(RandomSplitter((0.8, 0.2)))